<a href="https://colab.research.google.com/github/novoforce/Exploring-Pytorch/blob/master/new/1001_Pytorch_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# Create fully connected Neural Network

In [19]:
class NN(nn.Module):  #subclassing/inheriting from nn.Module
  def __init__(self,input_size,num_classes):  #input_size= 28x28 = 784
    super(NN,self).__init__() #calls the init function of the parent class(nn.Module)
    self.fc1= nn.Linear(input_size,50)
    self.fc2= nn.Linear(50,num_classes)

  def forward(self,x):
    x= F.relu(self.fc1(x)) #-->shape: (batch x 784)-->(batch x 50)
    x= self.fc2(x) #-->shape: (batch x 50)-->(batch x num_classes)
    return x

# Testing the class with some random generated values
# model= NN(784,10)
# x= torch.randn((64,784))
# print(model(x).shape)


# Set Device

In [20]:
device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters

In [26]:
input_size= 784
num_classes= 10
learning_rate= 0.001
batch_size= 64
num_epochs= 1

# Load Data

In [27]:
train_dataset= datasets.MNIST(root='datasets/',train=True,transform=transforms.ToTensor(),download=True)
train_loader= DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)

test_dataset= datasets.MNIST(root='datasets/',train=False,transform=transforms.ToTensor(),download=True)
test_loader= DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)

# Initialize network

In [28]:
model= NN(input_size=input_size,num_classes=num_classes).to(device)

# Loss & Optimizers

In [29]:
criterion= nn.CrossEntropyLoss()
optimizer= optim.Adam(model.parameters(),lr=learning_rate) 

# Train Network

In [31]:
for epoch in range(num_epochs):
    for batch_idx,(data,targets) in enumerate(train_loader):
        data= data.to(device)
        targets= targets.to(device)
        # print(data.shape) #here we have to reshape the input from (64x1x28x28) to (64x764) which is basically unrolling
        data= data.reshape(data.shape[0],-1)

        #forward propagation
        scores= model(data)
        loss= criterion(scores,targets)

        #backward propagation
        optimizer.zero_grad()
        loss.backward()

        #gradient descent
        optimizer.step()

# Check accuracy on training & test to see how good our model is

In [41]:
def check_accuracy(loader,model):
    num_correct= 0
    num_samples= 0
    model.eval()

    with torch.no_grad():
        if loader.dataset.train:
            print("Checking accuracy on Training data")
        else:
            print("Checking accuracy on Testing data")

        for x,y in loader:
            x= x.to(device)
            y= y.to(device)
            # print('y: ',y)
            x= x.reshape(x.shape[0],-1)
            scores= model(x)  #batch x num_classes   Here 'model' is the trained
            # print('scores: ',scores)
            _,predictions= scores.max(1)
            # print('vals: ',_,predictions)
            # break
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
        print(f'Got {num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f} %')

    model.train()

check_accuracy(train_loader,model)
check_accuracy(test_loader,model)


Checking accuracy on Training data
Got 56874/60000 with accuracy 94.79 %
Checking accuracy on Testing data
Got 9474/10000 with accuracy 94.74 %
